# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# custom package
from HelpHand import GetHand

In [ ]:
# custom algorithm
from Music_Recommender import MetaFilter

# Loading data

In [ ]:
tdf = pd.read_csv("/content/drive/MyDrive/data/files/MusicData/track_df.csv", index_col='idx')

In [ ]:
adf = pd.read_csv("/content/drive/MyDrive/data/files/MusicData/art_df.csv", index_col='idx')

# EDA

In [ ]:
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586601 entries, 0 to 586671
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586601 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586601 non-null  int64  
 3   artists           586601 non-null  object 
 4   id_artists        586601 non-null  object 
 5   danceability      586601 non-null  float64
 6   energy            586601 non-null  float64
 7   key               586601 non-null  int64  
 8   speechiness       586601 non-null  float64
 9   acousticness      586601 non-null  float64
 10  instrumentalness  586601 non-null  float64
 11  liveness          586601 non-null  float64
 12  valence           586601 non-null  float64
 13  year              586601 non-null  int64  
 14  decade            586601 non-null  int64  
dtypes: float64(7), int64(4), object(4)
memory usage: 71.6+ MB


In [ ]:
tdf.head(3)

,id,name,popularity,artists,id_artists,danceability,energy,key,speechiness,acousticness,instrumentalness,liveness,valence,year,decade
idx,,,,,,,,,,,,,,,
0,35iwgR4jXetI318WEWsa1Q,Carve,6,['Uli'],['45tIt06XoI0Iio4LBEVpls'],0.645,0.445,0,0.4510,0.674,0.7440,0.151,0.127,1922,1920
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],0.695,0.263,0,0.9570,0.797,0.0000,0.148,0.655,1922,1920
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0.434,0.177,1,0.0512,0.994,0.0218,0.212,0.457,1922,1920


We will convert 'artists' column's element to string

In [ ]:
# used custom function to clean artist column's data
tdf['artists'] = tdf['artists'].apply(GetHand.cleanString)

In [ ]:
tdf.sort_values(by=['year', 'popularity' ], ascending=False, inplace=True)

In [ ]:
# Converting name columns to lower case string

# tdf['name'] = tdf['name'].apply(lambda x : x.lower())
tdf['name'] = tdf['name'].str.lower()
tdf['artists'] = tdf['artists'].str.lower()

In [ ]:
tdf.head(2)

,id,name,popularity,artists,id_artists,danceability,energy,key,speechiness,acousticness,instrumentalness,liveness,valence,year,decade
idx,,,,,,,,,,,,,,,
93802,4iJyoBOLtHqaGxP12qzhQI,peaches (feat. daniel caesar & giveon),100,"justin bieber,daniel caesar,giveon","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",0.677,0.696,0,0.1190,0.321,0.000000,0.420,0.464,2021,2020
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99,olivia rodrigo,['1McMsnEElThX1knmY4oliG'],0.585,0.436,10,0.0601,0.721,0.000013,0.105,0.132,2021,2020


Handling duplicate values : As there has a lot of songs of different versions

In [ ]:
# tdf[tdf.duplicated(subset=['name'])]

In [ ]:
tdf.drop_duplicates(subset=['name','artists'], inplace=True)

#### Let's check some important column's correlation

In [ ]:
def checkCorr(df : pd.DataFrame, column:str):
  return df.corr()[[column]].sort_values(by = column, ascending=False)


In [ ]:
checkCorr(tdf, 'valence')

,valence
valence,1.000000
danceability,0.527957
energy,0.374499
speechiness,0.044991
key,0.018556
popularity,-0.000041
liveness,-0.000258
decade,-0.023847
year,-0.024486
instrumentalness,-0.175402


In [ ]:
checkCorr(tdf, 'acousticness')

,acousticness
acousticness,1.000000
instrumentalness,0.197041
speechiness,0.072827
liveness,-0.005860
key,-0.028427
valence,-0.182727
danceability,-0.240476
popularity,-0.365648
decade,-0.514297
year,-0.518662


In [ ]:
checkCorr(tdf, 'energy')

,energy
energy,1.000000
year,0.453243
decade,0.448521
valence,0.374499
popularity,0.300873
danceability,0.239762
liveness,0.127187
key,0.036898
speechiness,-0.059424
instrumentalness,-0.191091


In [ ]:
tdf.describe().T

,count,mean,std,min,25%,50%,75%,max
popularity,524209.0,27.332699,18.113126,0.0,13.0000,27.000000,40.00000,100.000
danceability,524209.0,0.563981,0.166526,0.0,0.4530,0.578000,0.68700,0.991
energy,524209.0,0.540538,0.251887,0.0,0.3420,0.547000,0.74600,1.000
key,524209.0,5.219664,3.519323,0.0,2.0000,5.000000,8.00000,11.000
speechiness,524209.0,0.109306,0.187824,0.0,0.0341,0.044600,0.07830,0.971
acousticness,524209.0,0.452709,0.347800,0.0,0.1010,0.430000,0.78500,0.996
instrumentalness,524209.0,0.113120,0.266574,0.0,0.0000,0.000023,0.00924,1.000
liveness,524209.0,0.215523,0.185993,0.0,0.0986,0.139000,0.28000,1.000
valence,524209.0,0.552108,0.257947,0.0,0.3460,0.564000,0.76900,1.000
year,524209.0,1988.894834,22.775068,1900.0,1974.0000,1993.000000,2008.00000,2021.000


In [ ]:
tdf.drop(tdf[tdf['artists'].str.contains("bts")].index, inplace=True)

In [ ]:
adf = adf.sort_values(by=['popularity', 'followers'], ascending=False)
adf['name'] = adf['name'].str.lower()
adf.drop(columns=['id'], inplace=True)
adf.head(3)

,followers,genres,name,popularity
idx,,,,
144481,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",justin bieber,100
313676,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",drake,98
126338,38869193.0,"['pop', 'post-teen pop']",taylor swift,98


In [ ]:
adf.describe().T

,count,mean,std,min,25%,50%,75%,max
followers,1162084.0,10220.703975,254399.492554,0.0,10.0,57.0,417.0,78900234.0
popularity,1162084.0,8.795974,13.557786,0.0,0.0,2.0,13.0,100.0


In [ ]:
adf[adf['popularity'].between(50,70)].describe().T

,count,mean,std,min,25%,50%,75%,max
followers,21188.0,201507.778884,362608.562475,2.0,16870.75,74427.0,223263.25,6394966.0
popularity,21188.0,56.580140,5.384612,50.0,52.00,55.0,60.00,70.0


In [ ]:
# removing artists name who has less than 1M followers (to make the dataset lite)

idx = adf[adf['followers']<1000000].index
adf.drop(idx, axis=0, inplace = True)

In [ ]:
adf.describe().T

,count,mean,std,min,25%,50%,75%,max
followers,1879.0,3.472167e+06,5.217450e+06,1000918.0,1309400.0,1895202.0,3371787.5,78900234.0
popularity,1879.0,7.264130e+01,8.102059e+00,26.0,67.0,72.0,78.0,100.0


In [ ]:
adf[adf['popularity'].between(50,100)].describe().T

,count,mean,std,min,25%,50%,75%,max
followers,1873.0,3.478902e+06,5.224425e+06,1000918.0,1310672.0,1905979.0,3376388.0,78900234.0
popularity,1873.0,7.273465e+01,7.932579e+00,50.0,68.0,72.0,78.0,100.0


# Algorithm

In [ ]:
songs = ['On my way by alan', 'faded by alan']

In [ ]:
mf = MetaFilter()
mf.fit(X = [tdf, adf])

,id,name,popularity,artists,id_artists,danceability,energy,key,speechiness,acousticness,instrumentalness,liveness,valence,year,decade


In [ ]:
pred = mf.predict(songs)

['year', 'popularity', 'valence', 'instrumentalness', 'acousticness', 'energy', 'danceability']


In [ ]:
pred

['hold on by justin bieber',
 '7 rings by ariana grande',
 'all of me by john legend',
 'snowman by sia',
 'better now by post malone',
 'f.n by lil tjay',
 'story of my life by one direction',
 'ghost by justin bieber',
 'cardigan by taylor swift',
 'call out my name by the weeknd',
 'wow. by post malone',
 'i fall apart by post malone',
 'ap - music from the film boogie by pop smoke',
 'i love you by billie eilish',
 'look at me! by xxxtentacion',
 'no idea by don toliver',
 'blueberry faygo by lil mosey',
 'dead man walking by brent faiyaz',
 'big gangsta by kevin gates',
 "still don't know my name by labrinth",
 'ransom by lil tecca',
 'murder on my mind by ynw melly',
 'deserve you by justin bieber',
 '2 much by justin bieber',
 'mr. perfectly fine (taylor’s version) (from the vault) by taylor swift',
 'toosie slide by drake',
 'si veo a tu mamá by bad bunny',
 'yo perreo sola by bad bunny',
 'haciendo que me amas by bad bunny',
 'levitating by dua lipa',
 'pov by ariana grande',


In [ ]:
mf.final_df.shape

(5534, 15)

# Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
ppl = Pipeline([('estimator', MetaFilter())])
ppl.fit(X = [tdf, adf])

Pipeline(steps=[('estimator', MetaFilter())])

In [ ]:
ppl._final_estimator.final_df

,id,name,popularity,artists,id_artists,danceability,energy,key,speechiness,acousticness,instrumentalness,liveness,valence,year,decade


In [ ]:
recom_songs = ppl.predict(['Havana by camila'])

['year', 'popularity', 'danceability', 'valence', 'instrumentalness', 'acousticness', 'energy']


In [ ]:
recom_songs

['rapstar by polo g',
 'te mudaste by bad bunny',
 'goosebumps by travis scott',
 '7 rings by ariana grande',
 'highest in the room by travis scott',
 'sicko mode by travis scott',
 'everything i wanted by billie eilish',
 'jocelyn flores by xxxtentacion',
 'shape of you by ed sheeran',
 'falling by harry styles',
 'in your eyes by the weeknd',
 "when the party's over by billie eilish",
 'la tóxica by farruko',
 'snowman by sia',
 'star shopping by lil peep',
 "ponte pa' mi by justin quiles",
 'hot girl bummer by blackbear',
 'maniac by conan gray',
 'willow by taylor swift',
 'explícito by myke towers',
 'everybody dies in their nightmares by xxxtentacion',
 'hawái by maluma',
 'say so by doja cat',
 'slow dancing in the dark by joji',
 'heat waves by glass animals',
 'sofia by clairo',
 'let me down slowly by alec benjamin',
 'booker t by bad bunny',
 "god's plan by drake",
 'thank u, next by ariana grande',
 'better now by post malone',
 '21 by polo g',
 'boss bitch by doja cat',
 '

In [ ]:
ppl._final_estimator.recommend_songs_df

,song,songPopu,artists,artistPopu
0,rapstar,89,polo g,91
1,te mudaste,88,bad bunny,98
2,goosebumps,88,travis scott,94
3,7 rings,87,ariana grande,95
4,highest in the room,87,travis scott,94
5,sicko mode,87,travis scott,94
6,everything i wanted,87,billie eilish,92
7,jocelyn flores,87,xxxtentacion,92
8,shape of you,87,ed sheeran,92
9,falling,87,harry styles,90
